In [1]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain.agents import AgentExecutor, create_react_agent, load_tools

from langchain_community.tools import ArxivQueryRun


In [2]:
from langchain_community.llms import Ollama

llm = Ollama(model="gemma2:2b", temperature=0.0)
llm



Ollama(model='gemma2:2b', temperature=0.0)

In [3]:
arxiv = ArxivAPIWrapper()
docs = arxiv.run("1605.08386")
docs

'Published: 2016-05-26\nTitle: Heat-bath random walks with Markov bases\nAuthors: Caprice Stanley, Tobias Windisch\nSummary: Graphs on lattice points are studied whose edges come from a finite set of\nallowed moves of arbitrary length. We show that the diameter of these graphs on\nfibers of a fixed integer matrix can be bounded from above by a constant. We\nthen study the mixing behaviour of heat-bath random walks on these graphs. We\nalso state explicit conditions on the set of moves so that the heat-bath random\nwalk, a generalization of the Glauber dynamics, is an expander in fixed\ndimension.'

In [4]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, load_tools
from langchain_community.llms import Ollama


tools = load_tools(
    ["arxiv"],
)
prompt = hub.pull("hwchase17/react")

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

k:\RAG Application\env\lib\site-packages\langsmith\client.py:5301: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


In [5]:
agent_executor.invoke({"input":"write something about the research paper of Attention all you need"})


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Thought: I need to find the research paper "Attention is All You Need" and summarize it. 
Action: arxiv
Action Input: Attention is All You NeedPublished: 2024-07-22
Title: Attention Is All You Need But You Don't Need All Of It For Inference of Large Language Models
Authors: Georgy Tyukin, Gbetondji J-S Dovonon, Jean Kaddour, Pasquale Minervini
Summary: The inference demand for LLMs has skyrocketed in recent months, and serving
models with low latencies remains challenging due to the quadratic input length
complexity of the attention layers. In this work, we investigate the effect of
dropping MLP and attention layers at inference time on the performance of
Llama-v2 models. We find that dropping dreeper attention layers only marginally
decreases performance but leads to the best speedups alongside dropping entire
layers. For example, removing 33\% of attention layers in a 13B Llama2 model
results in a 1.8\% drop in average performance over the OpenLLM benchmark. We
also observe that skip

{'input': 'write something about the research paper of Attention all you need',
 'output': ''}

In [6]:
# wikipedia searching using agents and wrapper classs
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
# Load tools, including Wikipedia search tool
tools2 = load_tools(
    ["wikipedia"],
)

# Pull the prompt from the hub
custom_prompt = hub.pull("hwchase17/react").partial(
    sistema="""You are a helpful AI assistant tasked with finding information.
    Your goal is to find information about the topic and provide clear summaries.
    
    
    Guidelines:
    1. Be specific in your queries
    2. Limit to the requested 500 count of words
    
    """
)
# Create the React agent
agent2 = create_react_agent(llm, tools, custom_prompt)

# Create the agent executor
agent_executor2 = AgentExecutor(agent=agent2, tools=tools2,handle_parsing_errors=True, max_iterations=3,  # Limit the number of iterations
    early_stopping_method="force" ,verbose=True)


In [7]:
agent_executor2.invoke({
    "input":"what is meant by Loss Function and how it is helpful in deep learning network trainning"
})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Thought: I need to understand loss functions and their role in training deep learning networks. 
Action: arxiv
Action Input: loss function in deep learningarxiv is not a valid tool, try one of [wikipedia].Thought:  I'll use Wikipedia to find information about Loss Functions in Deep Learning. 

Action: wikipedia
Action Input: Loss Function in Deep LearningPage: Reinforcement learning
Summary: Reinforcement learning (RL) is an interdisciplinary area of machine learning and optimal control concerned with how an intelligent agent ought to take actions in a dynamic environment in order to maximize the cumulative reward. Reinforcement learning is one of three basic machine learning paradigms, alongside supervised learning and unsupervised learning.
Q-learning at its simplest stores data in tables. This approach falters with increasing numbers of states/actions since the likelihood of the agent visiting a particular state and performing a particular action is increasingly small. 
Reinforcemen

{'input': 'what is meant by Loss Function and how it is helpful in deep learning network trainning',
 'output': "Loss functions play a crucial role in deep learning by providing a way to quantify the difference between the model's predictions and the actual target values. This difference, known as error or cost, is minimized during training to improve the model's accuracy.  Loss functions are essential for optimizing neural networks and enabling them to learn complex patterns from data."}

In [12]:
tools3=load_tools(["wikipedia","arxiv"],)

custom_prompt2 = hub.pull("hwchase17/react").partial(
    sistema="""You are a helpful AI assistant tasked with finding information.
    Your goal is to use right tool for searchiing .
    lets us say when the user is searching for general topics like "what is machine learning ?" you need to use wikipedia tool .
    when the user is searching for particular research papers or asking for research paper on particular topic you need use arxiv tool.

    
    
    Guidelines:
    1. Be specific in your queries
    2. Limit to the requested 500 count of words for wikipedia search .
    3. Limit to 3 research papers for reserch search .
    4. Apply Reverfication of your response.
    
    """
)
agent3 = create_react_agent(llm, tools3, custom_prompt2)

agent_executor3 = AgentExecutor(agent=agent3, tools=tools3,handle_parsing_errors=True, max_iterations=7,  # Limit the number of iterations
    early_stopping_method="force" ,verbose=True)




In [13]:
agent_executor3.invoke({
    "input":"Describe about the LinkedList Data Structure and its realtime use case"
})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Thought: To describe a LinkedList data structure, I need to explain what it is and then discuss its real-time use cases. 
Action: wikipedia
Action Input: LinkedList data structurePage: Linked data structure
Summary: In computer science, a linked data structure is a data structure which consists of a set of data records (nodes) linked together and organized by references (links or pointers).  The link between data can also be called a connector.
In linked data structures, the links are usually treated as special data types that can only be dereferenced or compared for equality.  Linked data structures are thus contrasted with arrays and other data structures that require performing arithmetic operations on pointers.  This distinction holds even when the nodes are actually implemented as elements of a single array, and the references are actually array indices: as long as no arithmetic is done on those indices, the data structure is essentially a linked one.
Linking can be done in two wa

{'input': 'Describe about the LinkedList Data Structure and its realtime use case',
 'output': 'Agent stopped due to iteration limit or time limit.'}

In [14]:
agent_executor3.invoke({
    "input":"find Top 3 Research papers about quantum computing."
})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Thought:  I need to search for top research papers on quantum computing. I can use Wikipedia or Arxiv for this. 
Action: arxiv
Action Input: "quantum computing" Published: 2022-08-01
Title: The Rise of Quantum Internet Computing
Authors: Seng W. Loke
Summary: This article highlights quantum Internet computing as referring to
distributed quantum computing over the quantum Internet, analogous to
(classical) Internet computing involving (classical) distributed computing over
the (classical) Internet. Relevant to quantum Internet computing would be areas
of study such as quantum protocols for distributed nodes using quantum
information for computations, quantum cloud computing, delegated verifiable
blind or private computing, non-local gates, and distributed quantum
applications, over Internet-scale distances.

Published: 2000-03-31
Title: Unconventional Quantum Computing Devices
Authors: Seth Lloyd
Summary: This paper investigates a variety of unconventional quantum computation
devices, i

{'input': 'find Top 3 Research papers about quantum computing.',
 'output': 'The top 3 research papers about quantum computing are:\n\n1. **The Rise of Quantum Internet Computing** by Seng W. Loke (2022-08-01) - This paper explores the concept of quantum internet computing and its potential applications in distributed quantum computing.\n2. **Unconventional Quantum Computing Devices** by Seth Lloyd (2000-03-31) - This paper investigates unconventional quantum computation devices, including fermionic quantum computers and those that exploit nonlinear quantum mechanics. \n3. **Geometrical perspective on quantum states and quantum computation** by Zeqian Chen (2013-11-20) - This paper presents a geometric model for quantum computation based on noncommutative geometry, offering an alternative approach to building a quantum computer.'}

In [15]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain.prompts import PromptTemplate
from langchain_community.llms import Ollama
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.tools import Tool




In [31]:
# Step 1: Define a custom prompt template for document retrieval

# Create a custom PromptTemplate
custom_prompt4 = hub.pull("hwchase17/react").partial(
  sistema=  """
You are a highly intelligent assistant specialized in answering questions using internal documents.

You have access to the following tools:
{tool_names}

Question: {input}

Provide a detailed response based on the available documents.

{agent_scratchpad}
"""

)

# Step 2: Set up the document retrieval tool
# Load internal documents from a directory (Specify the path to your internal documents)
loader = PyPDFLoader("Attention all you need.pdf")



In [20]:
# Create a vector store from the documents using embeddings
embeddings = OllamaEmbeddings(model="gemma2:2b")  # Use the desired embeddings (OpenAI or others)
doc_search = FAISS.from_documents(loader.load(), embeddings)  # Use FAISS for efficient retrieval

#

# Step 5: Create a React Agent with the custom prompt and document tool
# Here we create a React agent using the LLM and the single document tool defined above


In [32]:
 #Create a RetrievalQA tool using the vector store
retrieval_qa = RetrievalQA.from_chain_type(
    llm=Ollama(model="gemma2:2b"),  # Use the desired LLM model
    retriever=doc_search.as_retriever()
)

# Step 3: Define the document retrieval tool as a LangChain tool
document_tool = Tool(
    name="internal_document_search",
    func=retrieval_qa.run,  # Link to the document retrieval function
    description="Use this tool to search through internal documents and provide detailed information based on the content."
)

# Step 4: Specify the LLM configuration (e.g., Ollama model)
llm = Ollama(model="gemma2:2b")

In [40]:
document_tool

Tool(name='internal_document_search', description='Use this tool to search through internal documents and provide detailed information based on the content.', func=<bound method Chain.run of RetrievalQA(verbose=False, combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:"), llm=Ollama(model='gemma2:2b'), output_parser=StrOutputParser(), llm_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_variable_name='context'), retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores

In [33]:
agent = create_react_agent(llm, [document_tool], custom_prompt4)

# Step 6: Create the AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=[document_tool], verbose=True)



In [35]:
# Step 7: Example query for document retrieval


agent_executor.invoke({
    "input":" what is Meant by DecoderStack"
})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Thought: I need to search for information about "DecoderStack" in internal documents.
Action: internal_document_search
Action Input: "DecoderStack"The provided text describes the architecture of the Transformer, specifically focusing on the encoder and decoder stacks. 

**Here's a breakdown of the information and answer to your question:**

* **Encoder Stack:** The encoder has six identical layers stacked one upon another. Each layer contains two sub-layers:
    * Multi-head self-attention mechanism for capturing long-range dependencies.
    * A simple, position-wise fully connected feed-forward network. 
    * Residual connections and layer normalization are used to facilitate these sub-layers. 

* **Decoder Stack:** The decoder also has six identical layers stacked one upon another.  It uses the encoder outputs as input and adds a multi-head attention mechanism. 
   * This decoder stack incorporates an additional sub-layer that performs self-attention over the output of the encoder t

{'input': ' what is Meant by DecoderStack',
 'output': 'I provided a detailed explanation of the encoder and decoder stacks in the Transformer model, highlighting their individual structure and function. However, it is important to note that the document does not contain information about a "specific Decoder Stack". If you have any further questions related to the transformer or its architecture, feel free to ask!'}

In [44]:

custom_prompt5 = hub.pull("hwchase17/react").partial(
    sistema="""You are a helpful AI assistant tasked with finding information.
    Your goal is to use right tool for searchiing .
    lets us say when the user is searching for general topics like "what is machine learning ?" you need to use wikipedia tool .
    when the user is searching for particular research papers or asking for research paper on particular topic you need use arxiv tool.
    when the user is asked query about the internal Documnt use document_tool for querying context and generate response with help of context.

    
    
    Guidelines:
    1. Be specific in your queries and internal questions.
    2. Limit to the requested 1000 count of words for wikipedia search .
    3. Limit to 3 research papers for reserch search .
    4. Be concise the context retriver and generate good response.
    5. Apply Reverfication of your response.
    
    """
)



In [45]:
tools5 = load_tools(["wikipedia", "arxiv"])  

In [47]:
tools5.append(document_tool)

In [49]:
agent5 = create_react_agent(llm, tools5, custom_prompt5)

agent_executor5 = AgentExecutor(agent=agent5, tools=tools5,handle_parsing_errors=True, max_iterations=4,  # Limit the number of iterations
    early_stopping_method="force" ,verbose=True)

In [50]:
agent_executor5.invoke({
    "input":"what is meant by Encoder Stack in Attention Mechanism"
})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Thought: To understand what an encoder stack in an attention mechanism means, I need to access information about attention mechanisms and how they are implemented.  I can get this from Wikipedia or internal documents depending on the type of content I'm seeking. 
Action: wikipedia
Action Input: Encoder Stack Attention MechanismPage: Transformer (deep learning architecture)
Summary: A transformer is a deep learning architecture developed by researchers at Google and based on the multi-head attention mechanism, proposed in a 2017 paper "Attention Is All You Need". Text is converted to numerical representations called tokens, and each token is converted into a vector via looking up from a word embedding table. At each layer, each token is then contextualized within the scope of the context window with other (unmasked) tokens via a parallel multi-head attention mechanism allowing the signal for key tokens to be amplified and less important tokens to be diminished.
Transformers have the adv

{'input': 'what is meant by Encoder Stack in Attention Mechanism',
 'output': 'An encoder stack, a crucial component of Transformer models, uses multiple self-attention layers to capture relationships between words in a sentence. This process helps the model understand the context and meaning behind text, enabling efficient and accurate language processing tasks like machine translation.'}